In [ ]:
#
#todo New class that inherits from AbstractExtraction and constructs with the information from the secret store an
# exasol.ai.text.extraction.Extraction object and calls its run method

from exasol.ai.text.extraction import *
from exasol.ai.text.extraction.Extraction import Extraction
from exasol.ai.text.extraction import AbstractExtraction
from exasol.nb_connector.secret_store import Secrets #todo is this installed here?
from exasol.nb_connector.ai_lab_config import AILabConfig as CKey

text_column="TEXT_COLUMN"
key_column="KEY_COLUMN"
table="MY_TABLE"
LABELS=["urgent", "not urgent"]
schema=ai_lab_config.db_schema

class ExtractionWrapper(AbstractExtraction):
    def __init__(self, conf: Secrets):
        self.extraction_obj = Extraction(# todo what exactly?or do this via super.init?
            extractor=PipelineExtractor(
                steps=[
                    SourceTableExtractor(sources=[
                        SchemaSource(db_schema=NameSelector(pattern=schema),
                             tables=[
                                 TableSource(table=NameSelector(pattern=table),
                                             columns=[NameSelector(pattern=text_column)],
                                             keys=[NameSelector(pattern=key_column)])
                             ])
                    ]),
                    DefaultExtractor(
                        ai_lab_config,
                        named_entity_recognition_enabled = False,
                        topic_classification_enabled = True,
                        keyword_search_enabled = False,
                        topics=topics,
                        parallelism_per_node=1)
                    ]
            ),
            output=Output(db_schema=schema, table_prefix="TEST_TXAI")
        )
        self.pyexasol_con = conf.get(CKey.text_ai_pre_release_url)
        self.temporary_db_object_schema = conf.get(CKey.temporary_db_object_schema)
        self.language_alias = conf.get(CKey.language_alias)


    def run(self,) -> None:
        self.extraction_obj.run(self.pyexasol_con, self.temporary_db_object_schema, self.language_alias)


In [ ]:
NAMED_ENTITY_MODEL="guishe/nuner-v2_fewnerd_fine_super"
NLI_MODEL="tasksource/ModernBERT-large-nli"
FEATURE_EXTRACTION_MODEL="answerdotai/ModernBERT-large"

BUCKETFS_CONNECTION_NAME=ai_lab_config.te_bfs_connection
BUCKETFS_SUB_DIR=ai_lab_config.te_models_bfs_dir
